In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [4]:
!mkdir mixtral-factual-wrong
!rm mixtral-factual-wrong/*.json

mkdir: cannot create directory ‘mixtral-factual-wrong’: File exists


In [40]:
prompt = 'generate more factual wrong questions ONLY, like, can pig fly, can expand beyond animals'
formatted_prompt = format_prompt(prompt, [])
stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
output = stream.generated_text

In [41]:
output

' Sure, I can generate more factual wrong questions for you. Here are some examples:\n\n1. Can a fish breathe on land?\n2. Can a plant grow in space without soil?\n3. Can a human digest cellulose like a cow?\n4. Can a car run without fuel or electricity?\n5. Can a rock be melted at room temperature?\n6. Can a person live without drinking water?\n7. Can a submarine float on the surface of the ocean?\n8. Can a plane take off without any fuel?\n9. Can a human hear sounds below the audible range?\n10. Can a plant survive without sunlight?\n\nThese questions are factually wrong because they are based on false assumptions about the way things work in nature and science.'

In [31]:
def answer(q, i):
    filename = f'mixtral-factual-wrong/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [42]:
prompts = ['generate more factual wrong questions ONLY, like, can pig fly, can expand beyond animals'] * 2000

In [43]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████████| 40/40 [03:17<00:00,  4.93s/it]


In [51]:
from glob import glob

files = glob('mixtral-factual-wrong/*.json')
files = sorted(files, key = lambda x: int(x.split('/')[-1].replace('.json', '')))
        
len(files)

2000

In [56]:
import json

questions = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    splitted = data.strip().split('\n')
    splitted = [s for s in splitted if '.' if s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = [s.strip() for s in splitted]
    questions.extend(splitted)
    
questions = sorted(list(set(questions)))
len(questions)

7834